In [7]:
%%writefile vector_add.cu

#include <iostream>

__global__ void vectorAdd(const float* A, const float* B, float* C, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < N) {
        C[i] = A[i] + B[i];
    }
}

int main() {
    const int N = 10;
    float A[N], B[N], C[N];

    // Initialize host arrays
    for (int i = 0; i < N; ++i) {
        A[i] = i;
        B[i] = i * 2;
    }

    // Print initial values
    std::cout << "A = [";
    for (int i = 0; i < N; ++i) {
        std::cout << A[i] << (i == N - 1 ? "" : ", ");
    }
    std::cout << "]" << std::endl;

    std::cout << "B = [";
    for (int i = 0; i < N; ++i) {
        std::cout << B[i] << (i == N - 1 ? "" : ", ");
    }
    std::cout << "]" << std::endl;

    float *d_a, *d_b,*d_c;
    cudaMalloc(&d_a,N*sizeof(float));
    cudaMalloc(&d_b,N*sizeof(float));
    cudaMalloc(&d_c,N*sizeof(float));
    cudaMemcpy(d_a,A,N*sizeof(float),cudaMemcpyHostToDevice);
    cudaMemcpy(d_b,B,N*sizeof(float),cudaMemcpyHostToDevice);
    int blocksize=256;
    int gridsize=ceil(N/blocksize);
    vectorAdd<<<gridsize,blocksize>>>(d_a,d_b,d_c,N);
    cudaMemcpy(C,d_c,N*sizeof(float),cudaMemcpyDeviceToHost);

    // Print results
    std::cout << "C = [";
    for (int i = 0; i < N; ++i) {
        std::cout << C[i] << (i == N - 1 ? "" : ", ");
    }
    std::cout << "]" << std::endl;


    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}

Writing vector_add.cu


In [8]:
# Compile with the specified architecture
!nvcc vector_add.cu -o vector_add -gencode arch=compute_75,code=sm_75

# Run the executable
!./vector_add

A = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
B = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18]
C = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
